In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dense
import time

# Record the start time
start_time = time.time()

# Read data from Excel file
file_path = ''

df = pd.read_excel(file_path)

# Convert 'iranidate' to datetime and drop rows with invalid dates
df['iranidate'] = pd.to_datetime(df['iranidate'], errors='coerce')
df = df.dropna(subset=['iranidate'])

# Function to convert date values to numeric representation
def convert_to_numeric_date(date):
    base_date = datetime(2000, 1, 1)
    return (date - base_date).days

df['iranidate'] = df['iranidate'].apply(convert_to_numeric_date)

# Encode 'node' column
label_encoder = LabelEncoder()
df['node_encoded'] = label_encoder.fit_transform(df['node'])

# Handling NaN values by finding the nearest non-empty cells in the same column
for col in ['CO', 'O3', 'SO2', 'PM10', 'PM2.5', 'AQI']:
    for idx, value in df[col].items():
        if pd.isnull(value):  # Check for NaN values
            upper_cell = df[col].iloc[:idx].last_valid_index()  # Find the first non-empty cell above
            lower_cell = df[col].iloc[idx + 1:].first_valid_index()  # Find the first non-empty cell below
            if upper_cell is not None and lower_cell is not None:  # If both upper and lower cells are found
                avg = (df.at[upper_cell, col] + df.at[lower_cell, col]) / 2  # Calculate the average
                df.at[idx, col] = avg  # Fill the empty cell with the average


# Check sample threshold
your_threshold_value = 100
if len(df) < your_threshold_value:
    print("Insufficient samples in the dataset.")
else:
    output_columns = ['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5', 'AQI', 'Highest tempreture: 12pm', 'Wind:km/h']
    unique_nodes = df['node'].unique()

    
    for node_value in unique_nodes:
        node_df = df[df['node'] == node_value]
        for output_column in output_columns:
            if node_df[output_column].isnull().any():
                print(f"NaN values found in {output_column} for Node {node_value}. Handle missing values before modeling.")
            else:
                X = node_df[['iranidate', 'node_encoded']]
                Y = node_df[[output_column]]
                X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

                scaler = StandardScaler()
                X_train_scaled = scaler.fit_transform(X_train)
                X_test_scaled = scaler.transform(X_test)

                # Use LSTM model 
                model = Sequential([
                    LSTM(units=64, activation='relu', input_shape=(X_train_scaled.shape[1], 1)),
                    Dense(units=1)
                ])

                model.compile(optimizer='adam', loss='mean_squared_error')
                model.fit(X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1)),
                          Y_train, epochs=10, batch_size=32, verbose=0)

                Y_pred = model.predict(X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1)))
                Y_pred = np.clip(Y_pred, a_min=0, a_max=None)

                mse = mean_squared_error(Y_test, Y_pred)
                rmse = np.sqrt(mse)
                r2 = r2_score(Y_test, Y_pred)
                evs = explained_variance_score(Y_test, Y_pred)

                print(f"For Node {node_value} and output column {output_column}:")
                print(f"Mean Squared Error (MSE): {mse}") 
                print(f"Root Mean Squared Error (RMSE): {rmse}")
                print(f"R-squared Score: {r2}")
                print(f"Explained Variance Score: {evs}")

                # Record the end time
end_time = time.time()

# Calculate the duration of the program execution
duration_seconds = end_time - start_time

# Print the duration of the program execution
print(f"Duration of the program execution: {duration_seconds} seconds")

2024-02-26 00:11:36.444540: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


51/51 [==============================] - 0s 2ms/step
For Node 0 and output column CO:
Mean Squared Error (MSE): 160.88497830693066
Root Mean Squared Error (RMSE): 12.684044240971831
R-squared Score: 0.08329719369586375
Explained Variance Score: 0.08862456295365873
51/51 [==============================] - 0s 1ms/step
For Node 0 and output column O3:
Mean Squared Error (MSE): 544.7147470311608
Root Mean Squared Error (RMSE): 23.339124812879355
R-squared Score: 0.5754598586504394
Explained Variance Score: 0.5912664709790474
51/51 [==============================] - 0s 1ms/step
For Node 0 and output column NO2:
Mean Squared Error (MSE): 531.6284849666628
Root Mean Squared Error (RMSE): 23.057070173087098
R-squared Score: 0.02170116013986112
Explained Variance Score: 0.022240862641328518
51/51 [==============================] - 0s 1ms/step
For Node 0 and output column SO2:
Mean Squared Error (MSE): 264.9338892472808
Root Mean Squared Error (RMSE): 16.276789893811394
R-squared Score: 0.134191